#### The 2D and 3D poses are extracted using the official implementation of the  work done by Wenhao Li et al. "Exploiting Temporal Contexts with Strided Transformer for 3D Human Pose Estimation", IEEE Transactions on Multimedia, 2022.
#### The code and installation instructions can be found at:  https://github.com/Vegetebird/StridedTransformer-Pose3D 

### Code for visualization of the 2D skeletal data

In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt 
import matplotlib.gridspec as gridspec
from IPython.display import display, clear_output

In [5]:
from pathlib import Path

### Path to dataset ###
file_path = 'D:/Courses/thesis/data/21h9f9e30v9cl2fapjggz4q1x7/SitToStand/Data/STS_2D_skeletons_coarsened/Pt253_PD_n_698.csv'


In [6]:
# Load the CSV file and check its shape
# Load the CSV file and skip the header row
keypoints_csv = np.loadtxt(file_path, delimiter=',', skiprows=1)
print("Original CSV Shape:", keypoints_csv.shape)
print(keypoints_csv[:5])  # Show the first 5 rows to confirm data format
# Extract only the necessary columns (50 columns for 25 joints with x,y coordinates)
# Skipping the first column (frame number) and the second column (time)
keypoints_data = keypoints_csv[:, 2:52]
print("Filtered Data Shape:", keypoints_data.shape)

# Reshape the data correctly
number_of_joints = 25
dimension = 2
keypoints = keypoints_data.reshape(keypoints_data.shape[0], number_of_joints, dimension)
print("Reshaped Keypoints Shape:", keypoints.shape)
print(keypoints[:5])  # Show the first 5 rows to confirm data format

Original CSV Shape: (510, 52)
[[0.00000e+00 0.00000e+00 3.08000e+02 1.14000e+02 3.06000e+02 1.22000e+02
  2.90000e+02 1.24000e+02 2.86000e+02 1.48000e+02 3.06000e+02 1.40000e+02
  3.18000e+02 1.20000e+02 3.18000e+02 1.44000e+02 3.00000e+02 1.40000e+02
  3.06000e+02 1.58000e+02 2.96000e+02 1.58000e+02 2.96000e+02 1.78000e+02
  2.98000e+02 2.12000e+02 3.16000e+02 1.58000e+02 3.22000e+02 1.76000e+02
  3.16000e+02 2.10000e+02 3.06000e+02 1.10000e+02 3.08000e+02 1.10000e+02
  2.98000e+02 1.10000e+02 3.16000e+02 1.10000e+02 3.16000e+02 2.22000e+02
  3.22000e+02 2.20000e+02 3.16000e+02 2.12000e+02 2.98000e+02 2.22000e+02
  2.94000e+02 2.22000e+02 3.02000e+02 2.14000e+02]
 [1.00000e+00 3.33333e-02 3.08000e+02 1.14000e+02 3.06000e+02 1.22000e+02
  2.90000e+02 1.24000e+02 2.86000e+02 1.48000e+02 3.06000e+02 1.40000e+02
  3.18000e+02 1.20000e+02 3.18000e+02 1.44000e+02 3.00000e+02 1.40000e+02
  3.06000e+02 1.58000e+02 2.96000e+02 1.58000e+02 2.96000e+02 1.78000e+02
  2.98000e+02 2.12000e+02 3.160

## Draw 2D plot

In [7]:
# Define joint connections for the 25-joint STS data
connections = [(17, 15), (15, 0), (0, 16), (16, 18), (0, 1), (1, 5), (5, 6), (6, 7), (1, 2), (2, 3), (3, 4),
               (1, 8), (8, 12), (12, 13), (13, 14), (14, 19), (19, 20), (14, 21), (8, 9), (9, 10), (10, 11),
               (11, 22), (22, 23), (11, 24)]

# Define colors for visualization
lcolor = (255, 0, 0)  # Red for left side
rcolor = (0, 0, 255)  # Blue for right side
joint_color = (0, 255, 0)  # Green for joints
base_thickness = 1

# Loop through each frame to visualize the skeleton
for idx, kps in enumerate(keypoints):
    clear_output(wait=True)

    max_dim = round(np.max(kps) * 1.3)
    img = np.zeros((max_dim, max_dim, 3), dtype='uint8')

    for i, c in enumerate(connections):
        start = tuple(map(int, kps[c[0]]))
        end = tuple(map(int, kps[c[1]]))

        # Skip if either start or end is (0,0)
        if start == (0, 0) or end == (0, 0):
            continue

        # Determine the thickness of the lines
        thickness = 2 if i in [8] else base_thickness

        # Draw lines and joints
        cv2.line(img, start, end, lcolor if i % 2 == 0 else rcolor, thickness)
        cv2.circle(img, start, radius=3, color=joint_color, thickness=-1)
        cv2.circle(img, end, radius=3, color=joint_color, thickness=-1)

    plt.figure(figsize=(6, 6))
    plt.axis('off')
    plt.imshow(img)
    plt.title(f'Frame {idx + 1}/{len(keypoints)}')

    plt.rcParams['figure.dpi'] = 120
    plt.show()

In [8]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from IPython.display import clear_output
import imageio
import matplotlib

# Define the output GIF filename
output_gif = 'skeleton_animation.gif'
frames = []

# Define joint connections for the 25-joint STS data
connections = [(17, 15), (15, 0), (0, 16), (16, 18), (0, 1), (1, 5), (5, 6), (6, 7), (1, 2), (2, 3), (3, 4),
               (1, 8), (8, 12), (12, 13), (13, 14), (14, 19), (19, 20), (14, 21), (8, 9), (9, 10), (10, 11),
               (11, 22), (22, 23), (11, 24)]

# Define colors for visualization
lcolor = (255, 0, 0)  # Red for left side
rcolor = (0, 0, 255)  # Blue for right side
joint_color = (0, 255, 0)  # Green for joints
base_thickness = 1

# Loop through each frame to visualize the skeleton
for idx, kps in enumerate(keypoints):
    clear_output(wait=True)

    max_dim = round(np.max(kps) * 1.3)
    img = np.zeros((max_dim, max_dim, 3), dtype='uint8')

    for i, c in enumerate(connections):
        start = tuple(map(int, kps[c[0]]))
        end = tuple(map(int, kps[c[1]]))

        # Skip if either start or end is (0,0)
        if start == (0, 0) or end == (0, 0):
            continue

        # Determine the thickness of the lines
        thickness = 2 if i in [8] else base_thickness

        # Draw lines and joints
        cv2.line(img, start, end, lcolor if i % 2 == 0 else rcolor, thickness)
        cv2.circle(img, start, radius=3, color=joint_color, thickness=-1)
        cv2.circle(img, end, radius=3, color=joint_color, thickness=-1)

    plt.figure(figsize=(6, 6))
    plt.axis('off')
    plt.imshow(img)
    plt.title(f'Frame {idx + 1}/{len(keypoints)}')
    plt.show()
    # Save the current frame as an image to add to the GIF
    plt.savefig(f'frame_{idx}.png')
    plt.close()

    # Read the saved image and append to frames for GIF creation
    frames.append(imageio.v2.imread(f'frame_{idx}.png'))

# Save the frames as an animated GIF
imageio.mimsave(output_gif, frames, fps=10)
print(f"Animation saved as {output_gif}")

Animation saved as skeleton_animation.gif
